In [7]:
import numpy as np

#Original Solution
left = 0
right = 2 * np.pi
s_time = 0
e_time = 10

Times = np.array([64,128])*(e_time-s_time)

In [8]:
SAV2_dif = [0, 0]
now = 0
for times in Times:
    Nx = times
    Ny = times
    x = np.linspace(left,right,Nx,endpoint = False)
    y = np.linspace(left,right,Ny,endpoint = False)
    X, Y = np.meshgrid(x,y)

    phi_init = phi0(X,Y)
    C0 = 0
    C1 = 300

    (mE_SAV2,E_SAV2,T) = SAV2(phi_init,Nx,Ny,times,s_time,e_time,left,right,C0)
    SAV2_dif[now] = np.max(np.abs(mE_SAV2-E_SAV2))
    
    now += 1
print(f'SAV: {SAV2_dif[0]}\t{SAV2_dif[1]}')


C:\Users\LimJC\AppData\Local\Temp\ipykernel_6976\4252133249.py:2: RuntimeWarning: divide by zero encountered in divide
  return 0.05*np.sin(X)/np.sin(Y)
C:\Users\LimJC\AppData\Local\Temp\ipykernel_6976\4252133249.py:2: RuntimeWarning: invalid value encountered in divide
  return 0.05*np.sin(X)/np.sin(Y)
C:\Users\LimJC\AppData\Local\Temp\ipykernel_6976\4252133249.py:19: RuntimeWarning: invalid value encountered in subtract
  C[:,1:] -= v[:,:-1]
C:\Users\LimJC\AppData\Local\Temp\ipykernel_6976\4252133249.py:20: RuntimeWarning: invalid value encountered in subtract
  C[:,:-1] -= v[:,1:]
C:\Users\LimJC\AppData\Local\Temp\ipykernel_6976\4252133249.py:60: RuntimeWarning: invalid value encountered in subtract
  B = f(phi0)-2*phi0/dt


KeyboardInterrupt: 

In [ ]:
ESAV2_dif = [0, 0]
for times in Times:
    Nx = times
    Ny = times
    x = np.linspace(left,right,Nx,endpoint = False)
    y = np.linspace(left,right,Ny,endpoint = False)
    X, Y = np.meshgrid(x,y)

    phi_init = phi0(X,Y)
    C1 = 300
    
    (mE_ESAV2,E_ESAV2,T) = ESAV2(phi_init,Nx,Ny,times,s_time,e_time,left,right,C1)
    ESAV2_dif[now] = np.max(np.abs(mE_ESAV2-E_ESAV2))
    
    now += 1
print(f'E-SAV: {ESAV2_dif[0]}\t{ESAV2_dif[1]}')

In [ ]:
NSAVb2_dif = [0, 0]

for times in Times:
    Nx = times
    Ny = times
    x = np.linspace(left,right,Nx,endpoint = False)
    y = np.linspace(left,right,Ny,endpoint = False)
    X, Y = np.meshgrid(x,y)

    phi_init = phi0(X,Y)
    C0 = 0
    
    (mE_NSAVb2,E_NSAVb2,T) = NSAVb2(phi_init,Nx,Ny,times,s_time,e_time,left,right,C0)
    NSAVb2_dif[now] = np.max(np.abs(mE_NSAVb2-E_NSAVb2))
    
    now += 1

print(f'New SAV-B: {NSAVb2_dif[0]}\t{NSAVb2_dif[1]}')

In [ ]:
O1 = np.log2(SAV2_dif[0]/ SAV2_dif[1])
O2 = np.log2(ESAV2_dif[0]/ ESAV2_dif[1])
O3 = np.log2(NSAVb2_dif[0]/ NSAVb2_dif[1])

In [ ]:
print(f'SAV: {O1}')
print(f'ESAV: {O2}')
print(f'NSAVb2: {O3}')

In [5]:
globals().clear()

In [6]:
def phi0(X,Y):
    return 0.05*np.sin(X)*np.sin(Y)

def F(phi):
    temp = phi*phi -1
    return 25 * temp*temp

def f(phi):
    return 100 * phi * (phi*phi - 1)

def L1(v,dx,dy):#the minus laplacian operator for periodic boundary condition
    B = 2 * v
    B[1:] -= v[:-1]
    B[:-1] -= v[1:]
    B[0] -= v[-1]
    B[-1] -= v[0]
    B /= dy*dy
    C = 2 * v
    C[:,1:] -= v[:,:-1]
    C[:,:-1] -= v[:,1:]
    C[:,0] -= v[:,-1]
    C[:,-1] -= v[:,0]
    C /= dx*dx
    return B+C

def v_prod(u,v,dx,dy):
    return dx*dy*np.vdot(u,v)

def Energy(phi,dx,dy):
    return 0.5*v_prod(phi,L1(phi,dx,dy),dx,dy)+dx*dy*np.sum(F(phi))

#2nd order SAV
def SAV2(phi_init,Nx,Ny,times,s_time, e_time, left, right, C0):
    dt = (e_time-s_time)/times
    T = s_time + np.arange(times)*dt
    x,dx = np.linspace(left,right,Nx,retstep=True, endpoint=False)
    y,dy = np.linspace(left,right,Ny,retstep=True, endpoint=False)
    X,Y = np.meshgrid(x,y)

    # initial value
    phi0 = phi_init
    E1 = dx*dy*np.sum(F(phi0))
    E = [v_prod(phi0,L1(phi0,dx,dy),dx,dy)/2 + E1]
    E_bar = E.copy()
    r0 = np.sqrt(E1+C0)
    m = np.arange(Nx//2+1)
    n = np.arange(Ny)

    #eigenvalue of RFFT 2d
    l1 = 2 / dx * np.sin(m * np.pi / Nx)
    l1 = - l1 * l1
    l2 = 2 / dy * np.sin(n * np.pi / Ny)
    l2 = - l2 * l2

    c = 2/dt
    l1, l2 = np.meshgrid(l1,l2)
    l = l1 + l2 - c #eigenvalue of (-L-1/dt)(-(I-dtGL)/dt)
    #Solve
    for t in T:
        B = f(phi0)-2*phi0/dt
        B_hat = np.fft.rfft2(B)
        temp = B_hat/l
        phi_bar = np.fft.irfft2(temp)
        
        temp = dx*dy*np.sum(F(phi_bar))
        B = f(phi_bar)/np.sqrt(temp+C0)
        v = -B/2
        C = 2*B*r0+v*v_prod(B,phi0,dx,dy)-2*phi0/dt+L1(phi0,dx,dy)

        B_hat = np.fft.rfft2(B)
        temp = B_hat/l
        b = np.fft.irfft2(temp)
        C_hat = np.fft.rfft2(C)
        temp = C_hat/l
        gamma = np.fft.irfft2(temp)

        phi1 = gamma - v_prod(v,gamma,dx,dy) /(1+v_prod(v,b,dx,dy))*b
        r1 = v_prod(B,phi1-phi0,dx,dy)/2 + r0
        
        E0 = v_prod(phi1,L1(phi1,dx,dy),dx,dy)/2
        E1 = dx*dy*np.sum(F(phi1))
        E.append(E0 + E1)
        E_bar.append(E0 + r1 * r1 - C0)
        phi0 = phi1
        r0 = r1
        
    return (np.array(E_bar),np.array(E),T)

#Stabilized Preditor-Corrector Method, M=-Delta
def SPC(phi_init,Nx,Ny,times,s_time, e_time, left, right,L):
    dt = (e_time-s_time)/times
    T = s_time + np.arange(times)*dt
    x,dx = np.linspace(left,right,Nx,retstep=True, endpoint=False)
    y,dy = np.linspace(left,right,Ny,retstep=True, endpoint=False)
    X,Y = np.meshgrid(x,y)

    # initial value
    phi0 = phi_init
    phi = [phi0]
    E1 = dx*dy*np.sum(F(phi0))
    E = [v_prod(phi[0],L1(phi0,dx,dy),dx,dy)/2 + E1]
    E_bar = E.copy()
    m = np.arange(Nx//2+1)
    n = np.arange(Ny)
    
    LL = L*L
    
    #eigenvalue of RFFT 2d
    l1 = 2 / dx * np.sin(m * np.pi / Nx)
    l1 = - l1 * l1
    l2 = 2 / dy * np.sin(n * np.pi / Ny)
    l2 = - l2 * l2

    c = 2/dt + LL
    l1, l2 = np.meshgrid(l1,l2)
    l = l1 + l2 - c #eigenvalue of (-L-2/dt-L^2)(-(I-dtGL)/dt)

    for t in T:
        b = f(phi0)-2*phi0/dt-LL*phi0
        b_hat = np.fft.rfft2(b)
        temp = b_hat/l
        phi_h = np.fft.irfft2(temp)
        
        b = 2*f(phi_h)-2*phi0/dt - 2*LL*phi_h + L1(phi0,dx,dy)+LL*phi0
        b_hat = np.fft.rfft2(b)
        temp = b_hat/l
        phi1 = np.fft.irfft2(temp)
        
        phi.append(phi1)
        
        E0 = v_prod(phi1,L1(phi1,dx,dy),dx,dy)/2
        E1 = dx*dy*np.sum(F(phi1))
        E.append(E0 + E1)
        phi0 = phi1
        
    return (np.array(E),T)

def ESAV2(phi_init,Nx,Ny,times,s_time,e_time,left,right,C0):
    dt = (e_time-s_time)/times
    T = np.arange(times)*dt
    x,dx = np.linspace(left,right,Nx,retstep=True, endpoint=False)
    y,dy = np.linspace(left,right,Ny,retstep=True, endpoint=False)
    X,Y = np.meshgrid(x,y)

    # initial value
    phi0 = phi_init

    E1 = dx*dy * np.sum(F(phi0))
    E = [v_prod(phi0,L1(phi0,dx,dy),dx,dy)/2 + E1]
    E_bar = E.copy()
    s0 = np.exp(E1/C0)
    
    m = np.arange(Nx//2+1)
    n = np.arange(Ny)
    #eigenvalue of RFFT 2d
    l1 = 2 / dx * np.sin(m * np.pi / Nx)
    l1 = - l1 * l1
    l2 = 2 / dy * np.sin(n * np.pi / Ny)
    l2 = - l2 * l2

    c = 2/dt
    l1, l2 = np.meshgrid(l1,l2)
    l = l1 + l2 - c #eigenvalue of (-L-2/dt)(-(2-dtGL)/dt)
    
    #Inverse of (1-\frac12dt)
    B = f(phi0)-2*phi0/dt
    B_hat = np.fft.rfft2(B)
    temp = B_hat/l
    phi_half = np.fft.irfft2(temp)
    
    s_half = np.exp(dx*dy*np.sum(F(phi_half))/C0)
    b = f(phi_half)
    #Solve
    for t in T:
        B = -2*phi0/dt+2*b+L1(phi0,dx,dy)
        B_hat = np.fft.rfft2(B)
        temp = B_hat/l
        phi1 = np.fft.irfft2(temp)
        
        s1 = s0*np.exp(v_prod(b,phi1-phi0,dx,dy)/C0)
        E1 = dx*dy*np.sum(F(phi1))
        E0 = v_prod(phi1,L1(phi1,dx,dy),dx,dy)/2
        E.append(E0 + E1)

        phi_half = (3*phi1-phi0)/2
        phi0 = phi1
        s_half = (3*s1-s0)/2
        s0 = s1
        
        E_bar.append(E0 + C0*np.log(s1))
        b = s_half*np.exp(-dx*dy*np.sum(F(phi_half))/C0) * f(phi_half)
        
    return (np.array(E_bar),np.array(E),T)

def NSAVb2(phi_init,Nx,Ny,times,s_time,e_time,left,right,C0):
    dt = (e_time-s_time)/times
    T = s_time + np.arange(times)*dt
    x, dx = np.linspace(left,right,Nx,retstep = True, endpoint = False)
    y, dy = np.linspace(left,right,Ny,retstep = True, endpoint = False)
    
    X,Y = np.meshgrid(x,y)

    # initial value
    phi = [phi_init]
    phi0 = phi_init
    E = [Energy(phi0,dx,dy)]
    E_bar = E.copy()
    C = 10
    q = E[0] + C0
    xi = 1
    m = np.arange(Nx//2+1)
    n = np.arange(Ny)

    #eigenvalue of RFFT 2d
    l1 = 2 / dx * np.sin(m * np.pi / Nx)
    l1 = - l1 * l1
    l2 = 2 / dy * np.sin(n * np.pi / Ny)
    l2 = - l2 * l2

    c = 2/dt
    l1, l2 = np.meshgrid(l1,l2)
    l = l1 + l2 - c #eigenvalue of (-L-1/dt)

    #Solve
    for t in T:
        b_hat = np.fft.rfft2(xi*f(phi0)-2*phi0/dt)
        phi_hat = b_hat/l
        phi_h = np.fft.irfft2(phi_hat)
        mu = L1(phi_h,dx,dy)+xi*f(phi0)
        E_h = Energy(phi_h,dx,dy)
        q_h = 2*q/(2+dt/E_h*v_prod(mu,mu,dx,dy))
        xi_h = (q_h-C0)/E_h
        
        b_hat = np.fft.rfft2(2*xi_h*f(phi0) - 2*phi0/dt - L1(phi0,dx,dy))
        phi_hat = b_hat/l
        phi1 = np.fft.irfft2(phi_hat)
        mu = L1((phi0+phi1)/2,dx,dy)+xi_h*f(phi_h)
        temp = dt/2/E_h*v_prod(mu,mu,dx,dy)
        q = (1-temp)/(1+temp)*q

        E.append(Energy(phi1,dx,dy))
        E_bar.append(q-C0)
        phi0 = phi1
        phi.append(phi1)
        xi = (q-C0)/E[-1]

    return (np.array(E_bar),np.array(E),T)